## 1 Show Info

In [11]:
!nvidia-smi

Mon Jan 20 05:28:48 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0              33W /  70W |   4511MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [17]:
!pwd

/content


In [12]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
stepping	: 3
microcode	: 0xffffffff
cpu MHz		: 2000.192
cache size	: 39424 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx avx512f avx512dq rdseed adx smap clflushopt clwb avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmi

## 2 Clone Repo

In [1]:
!cd /content
!rm -rf sample_data ChatTTS
!git clone https://github.com/2noise/ChatTTS.git

Cloning into 'ChatTTS'...
remote: Enumerating objects: 2685, done.
remote: Counting objects: 100% (765/765), done.
remote: Compressing objects: 100% (312/312), done.
remote: Total 2685 (delta 538), reused 454 (delta 453), pack-reused 1920 (from 3)
Receiving objects: 100% (2685/2685), 8.02 MiB | 12.76 MiB/s, done.
Resolving deltas: 100% (1608/1608), done.


## 3 Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 4 Install Testset

In [ ]:
!tar -xvf /content/drive/MyDrive/Colab\ Notebooks/seedtts_testset.tar

In [29]:
Audio(filename="/content/seedtts_testset/zh/wavs/10002287-00000095.wav", rate=24_000, autoplay=True)

## 5 Install Requirements

In [ ]:
!pip install -r /content/ChatTTS/requirements.txt
!ldconfig /usr/lib64-nvidia

## 6 Import Packages

In [30]:
import torch

torch._dynamo.config.cache_size_limit = 64
torch._dynamo.config.suppress_errors = True
torch.set_float32_matmul_precision("high")

from ChatTTS import ChatTTS
from ChatTTS.tools.logger import get_logger
from ChatTTS.tools.normalizer import normalizer_en_nemo_text, normalizer_zh_tn
from IPython.display import Audio

import time

## 7 Load Models

In [5]:
logger = get_logger("ChatTTS", format_root=True)
chat = ChatTTS.Chat(logger)

# try to load normalizer
try:
    chat.normalizer.register("en", normalizer_en_nemo_text())
except ValueError as e:
    logger.error(e)
except:
    logger.warning("Package nemo_text_processing not found!")
    logger.warning(
        "Run: conda install -c conda-forge pynini=2.1.5 && pip install nemo_text_processing",
    )
try:
    chat.normalizer.register("zh", normalizer_zh_tn())
except ValueError as e:
    logger.error(e)
except:
    logger.warning("Package WeTextProcessing not found!")
    logger.warning(
        "Run: conda install -c conda-forge pynini=2.1.5 && pip install WeTextProcessing",
    )

 NeMo-text-processing :: INFO     :: Creating ClassifyFst grammars.
[+0000 20250120 05:15:44] [INFO] NeMo-text-processing | tokenize_and_classify | Creating ClassifyFst grammars.
2025-01-20 05:16:19,973 WETEXT INFO found existing fst: /usr/local/lib/python3.11/dist-packages/tn/zh_tn_tagger.fst
[+0000 20250120 05:16:19] [INFO] wetext-zh_normalizer | processor | found existing fst: /usr/local/lib/python3.11/dist-packages/tn/zh_tn_tagger.fst
2025-01-20 05:16:19,978 WETEXT INFO                     /usr/local/lib/python3.11/dist-packages/tn/zh_tn_verbalizer.fst
[+0000 20250120 05:16:19] [INFO] wetext-zh_normalizer | processor |                     /usr/local/lib/python3.11/dist-packages/tn/zh_tn_verbalizer.fst
2025-01-20 05:16:19,981 WETEXT INFO skip building fst for zh_normalizer ...
[+0000 20250120 05:16:19] [INFO] wetext-zh_normalizer | processor | skip building fst for zh_normalizer ...


### Here are three choices for loading models,

#### 1. Load models from Hugging Face (recommend)

In [6]:
# use force_redownload=True if the weights have been updated.
chat.load(source="huggingface")

[+0000 20250120 05:19:36] [INFO] ChatTTS | core | download from HF: https://huggingface.co/2Noise/ChatTTS
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

DVAE.safetensors:   0%|          | 0.00/60.4M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/853M [00:00<?, ?B/s]

Embed.safetensors:   0%|          | 0.00/146M [00:00<?, ?B/s]

...

config/gpt.yaml:   0%|          | 0.00/346 [00:00<?, ?B/s]

config/path.yaml:   0%|          | 0.00/309 [00:00<?, ?B/s]

config/vocos.yaml:   0%|          | 0.00/460 [00:00<?, ?B/s]

[+0000 20250120 05:20:01] [INFO] ChatTTS | core | use device cuda:0
[+0000 20250120 05:20:01] [INFO] ChatTTS | core | vocos loaded.
[+0000 20250120 05:20:02] [INFO] ChatTTS | core | dvae loaded.
[+0000 20250120 05:20:02] [INFO] ChatTTS | core | embed loaded.
[+0000 20250120 05:20:02] [INFO] ChatTTS | core | gpt loaded.
[+0000 20250120 05:20:02] [INFO] ChatTTS | core | speaker loaded.
[+0000 20250120 05:20:03] [INFO] ChatTTS | core | decoder loaded.
[+0000 20250120 05:20:03] [INFO] ChatTTS | core | tokenizer loaded.


True

#### 2. Load models from local directories 'asset' and 'config'

In [ ]:
chat.load()
# chat.load(source='local') same as above

#### 3. Load models from a custom path

In [ ]:
# write the model path into custom_path
chat.load(source="custom", custom_path="YOUR CUSTOM PATH")

### You can also unload models to save the memory

In [ ]:
chat.unload()

## Inference

### Batch infer

In [7]:
texts = [
    "So we found being competitive and collaborative was a huge way of staying motivated towards our goals, so one person to call when you fall off, one person who gets you back on then one person to actually do the activity with.",
] * 3 + [
    "我觉得像我们这些写程序的人，他，我觉得多多少少可能会对开源有一种情怀在吧我觉得开源是一个很好的形式。现在其实最先进的技术掌握在一些公司的手里的话，就他们并不会轻易的开放给所有的人用。"
] * 3

wavs = chat.infer(texts)

text:   0%|          | 1/384(max) [00:01,  1.12s/it]We detected that you are passing `past_key_values` as a tuple of tuples. This is deprecated and will be removed in v4.47. Please convert your cache or use an appropriate `Cache` class (https://huggingface.co/docs/transformers/kv_cache#legacy-cache-format)
text:  28%|██▊       | 107/384(max) [00:03, 29.43it/s]
code:  43%|████▎     | 890/2048(max) [00:22, 39.92it/s]


In [8]:
Audio(wavs[0], rate=24_000, autoplay=True)

In [9]:
Audio(wavs[3], rate=24_000, autoplay=True)

### Custom params

In [10]:
params_infer_code = ChatTTS.Chat.InferCodeParams(
    prompt="[speed_5]",
    temperature=0.3,
)
params_refine_text = ChatTTS.Chat.RefineTextParams(
    prompt="[oral_2][laugh_0][break_6]",
)

wav = chat.infer(
    "四川美食可多了，有麻辣火锅、宫保鸡丁、麻婆豆腐、担担面、回锅肉、夫妻肺片等，每样都让人垂涎三尺。",
    params_refine_text=params_refine_text,
    params_infer_code=params_infer_code,
)

[+0000 20250120 05:25:32] [INFO] ChatTTS | norm | replace homophones: 涎->闲
text:  14%|█▍        | 53/384(max) [00:01, 46.16it/s]
code:  20%|██        | 412/2048(max) [00:09, 43.09it/s]


In [13]:
Audio(wav[0], rate=24_000, autoplay=True)

### fix random speaker

In [ ]:
rand_spk = chat.sample_random_speaker()
print(rand_spk)  # save it for later timbre recovery

params_infer_code = ChatTTS.Chat.InferCodeParams(
    spk_emb=rand_spk,
)

wav = chat.infer(
    "四川美食确实以辣闻名，但也有不辣的选择。比如甜水面、赖汤圆、蛋烘糕、叶儿粑等，这些小吃口味温和，甜而不腻，也很受欢迎。",
    params_infer_code=params_infer_code,
)

In [ ]:
Audio(wav[0], rate=24_000, autoplay=True)

### Zero shot (simulate speaker)

In [ ]:
! cd ChatTTS/

In [28]:
from ChatTTS.tools.audio import load_audio

spk_smp = chat.sample_audio_speaker(load_audio("sample.mp3", 24000))
print(spk_smp)  # save it in order to load the speaker without sample audio next time

params_infer_code = ChatTTS.Chat.InferCodeParams(
    spk_smp=spk_smp,
    txt_smp="与sample.mp3内容完全一致的文本转写。",
)

wav = chat.infer(
    "四川美食确实以辣闻名，但也有不辣的选择。比如甜水面、赖汤圆、蛋烘糕、叶儿粑等，这些小吃口味温和，甜而不腻，也很受欢迎。",
    params_infer_code=params_infer_code,
)

FileNotFoundError: File not found: sample.mp3

In [ ]:
Audio(wav[0], rate=24_000, autoplay=True)

### Two stage control

In [ ]:
text = "So we found being competitive and collaborative was a huge way of staying motivated towards our goals, so one person to call when you fall off, one person who gets you back on then one person to actually do the activity with."
refined_text = chat.infer(text, refine_text_only=True)
refined_text

In [ ]:
wav = chat.infer(refined_text, skip_refine_text=True)

In [ ]:
Audio(wav[0], rate=24_000, autoplay=True)

## LLM Call

In [ ]:
from ChatTTS.tools.llm import ChatOpenAI

API_KEY = ""
client = ChatOpenAI(
    api_key=API_KEY, base_url="https://api.deepseek.com", model="deepseek-chat"
)

In [ ]:
user_question = "四川有哪些好吃的美食呢?"

In [ ]:
text = client.call(user_question, prompt_version="deepseek")
text

In [ ]:
text = client.call(text, prompt_version="deepseek_TN")
text

In [ ]:
wav = chat.infer(text)

In [ ]:
Audio(wav[0], rate=24_000, autoplay=True)

## 性能测试

### 推理速度测试

In [ ]:
import time

generation_times = []

for idx, text in enumerate(test_texts):
    start_time = time.time()
    with torch.no_grad():
        speech = model.generate_speech(text.strip())
    end_time = time.time()
    generation_time = end_time - start_time
    generation_times.append(generation_time)
    print(f'生成第{idx+1}个语音花费时间: {generation_time:.2f}秒')

average_time = sum(generation_times) / len(generation_times)
print(f'平均生成时间: {average_time:.2f}秒/句')

### 资源占用情况

In [ ]:
# 查看GPU信息
!nvidia-smi

# 查看CPU和内存使用情况
!htop